# BIZ 프로젝트 : 부실기업 예측

## Step2 : 데이터 전처리

부실기업은 과거 3년간 연속해 이자보상 배율이 1.0미만인 기업을 의미하며  
이를 이용하여 타겟변수인 부실기업여부 변수를 생성

### 필요라이브러리 불러오기

In [453]:
import pandas as pd
import numpy as np

### 데이터 불러오기

In [454]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/clean_financial_data.csv", encoding='utf-8-sig', low_memory=False)

In [455]:
data.columns

Index(['업체코드', '종목코드', '종목명', '2020/총자산증가율', '2021/총자산증가율', '2022/총자산증가율',
       '2023/총자산증가율', '2024/총자산증가율', '2020/유형자산증가율', '2021/유형자산증가율',
       ...
       '2020/조세공과(구성비)', '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)',
       '2024/조세공과(구성비)', '2020/감가상각비(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2024/감가상각비(구성비)'],
      dtype='object', length=543)

21~23년도 데이터 선택

In [456]:
import pandas as pd

# 21~23년도 데이터만을 사용하기 위해 해당 연도가 포함된 열 필터링
years = ['2021', '2022', '2023']
cols_to_keep = ['업체코드', '종목코드', '종목명'] + [col for col in data.columns if any(year in col for year in years)]

# 필터링된 열들로 새로운 데이터프레임 생성
data = data[cols_to_keep]

# 결과 확인
print("필터링된 데이터프레임의 열들:")
print(data.columns)

필터링된 데이터프레임의 열들:
Index(['업체코드', '종목코드', '종목명', '2021/총자산증가율', '2022/총자산증가율', '2023/총자산증가율',
       '2021/유형자산증가율', '2022/유형자산증가율', '2023/유형자산증가율', '2021/유동자산증가율',
       ...
       '2023/금융비용(구성비)', '2021/임차료(구성비)', '2022/임차료(구성비)', '2023/임차료(구성비)',
       '2021/조세공과(구성비)', '2022/조세공과(구성비)', '2023/조세공과(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)'],
      dtype='object', length=327)


연도별 영업이익이자보상비율의 값이 결측인 행이 존재하는경우 해당 행 제거

In [457]:
import pandas as pd

# '영업이익이자보상비율' 이름이 들어가는 변수들 필터링
interest_coverage_ratio_cols = data.filter(like='영업이익이자보상비율').columns

# 필터링된 변수명 출력
print("영업이익이자보상비율 이름이 들어가는 변수들:")
print(interest_coverage_ratio_cols)

# 결측값이 존재하는 행 제거 전의 행 수
initial_row_count = len(data)

# 해당 값들에 대해서 결측이 존재하는 경우 해당 행 제거
data = data.dropna(subset=interest_coverage_ratio_cols)

# 결측값이 존재하는 행 제거 후의 행 수
final_row_count = len(data)

# 제거된 행의 갯수 출력
removed_rows = initial_row_count - final_row_count
print(f"\n제거된 행의 갯수: {removed_rows}")

영업이익이자보상비율 이름이 들어가는 변수들:
Index(['2021/영업이익이자보상비율', '2022/영업이익이자보상비율', '2023/영업이익이자보상비율'], dtype='object')

제거된 행의 갯수: 8335


In [458]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30304 entries, 0 to 38638
Columns: 327 entries, 업체코드 to 2023/감가상각비(구성비)
dtypes: float64(324), object(3)
memory usage: 75.8+ MB


부실기업여부 변수 생성  
3년 연속 영업이익이자보상비율 < 1 => 부실기업

In [459]:
# 2023/부실기업 변수 생성 및 값 할당
data['2023/부실기업'] = ((data['2021/영업이익이자보상비율'] < 1) & 
                        (data['2022/영업이익이자보상비율'] < 1) & 
                        (data['2023/영업이익이자보상비율'] < 1)).astype(int)

# 결과 확인
print("\n2023/부실기업 변수 값:")
print(data['2023/부실기업'].value_counts())


2023/부실기업 변수 값:
0    24097
1     6207
Name: 2023/부실기업, dtype: int64


In [460]:
data.columns

Index(['업체코드', '종목코드', '종목명', '2021/총자산증가율', '2022/총자산증가율', '2023/총자산증가율',
       '2021/유형자산증가율', '2022/유형자산증가율', '2023/유형자산증가율', '2021/유동자산증가율',
       ...
       '2021/임차료(구성비)', '2022/임차료(구성비)', '2023/임차료(구성비)', '2021/조세공과(구성비)',
       '2022/조세공과(구성비)', '2023/조세공과(구성비)', '2021/감가상각비(구성비)',
       '2022/감가상각비(구성비)', '2023/감가상각비(구성비)', '2023/부실기업'],
      dtype='object', length=328)

In [461]:
data.head(3)

,업체코드,종목코드,종목명,2021/총자산증가율,2022/총자산증가율,2023/총자산증가율,2021/유형자산증가율,2022/유형자산증가율,2023/유형자산증가율,2021/유동자산증가율,...,2021/임차료(구성비),2022/임차료(구성비),2023/임차료(구성비),2021/조세공과(구성비),2022/조세공과(구성비),2023/조세공과(구성비),2021/감가상각비(구성비),2022/감가상각비(구성비),2023/감가상각비(구성비),2023/부실기업
0,N350605,A000020,동화약품,2.70,3.27,10.23,10.41,7.64,15.13,-3.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,N320498,A000040,KR모터스,1.45,-0.52,-4.89,5.78,4.85,-10.98,-12.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,N320684,A000050,경방,0.15,-6.09,-1.40,-23.56,665.18,-4.12,-0.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


이때 종목코드 x -> 해당 기업이 상장되지 않은 경우

In [462]:
# '종목코드' 열의 값이 있으면 1, 없으면 0을 부여하는 새로운 열 생성
data['상장여부'] = data['종목코드'].notnull().astype(int)

# '종목코드' 열 제거
data = data.drop(columns=['종목코드'])

결측비율 계산

In [463]:
import pandas as pd

# 모든 변수들에 대해 결측 비율 계산
missing_ratio_all = data.isnull().mean()

# # 결측 비율이 0.5 이상인 변수들 필터링 및 출력
# cols_above_50 = missing_ratio_all[missing_ratio_all >= 0.5].index
# print("\n결측 비율이 0.5 이상인 변수들:")
# print(cols_above_50)

# # 결측 비율이 0.6 이상인 변수들 필터링 및 출력
# cols_above_60 = missing_ratio_all[missing_ratio_all >= 0.6].index
# print("\n결측 비율이 0.6 이상인 변수들:")
# print(cols_above_60)

# 결측 비율이 0.7 이상인 변수들 필터링 및 출력
cols_above_70 = missing_ratio_all[missing_ratio_all >= 0.7].index
print("\n결측 비율이 0.7 이상인 변수들:")
print(cols_above_70)

# 결측 비율이 0.8 이상인 변수들 필터링 및 출력
cols_above_80 = missing_ratio_all[missing_ratio_all >= 0.8].index
print("\n결측 비율이 0.8 이상인 변수들:")
print(cols_above_80)

# 결측 비율이 0.9 이상인 변수들 필터링 및 출력
cols_above_90 = missing_ratio_all[missing_ratio_all >= 0.9].index
print("\n결측 비율이 0.9 이상인 변수들:")
print(cols_above_90)


결측 비율이 0.7 이상인 변수들:
Index(['2023/감가상각비/총비용비율', '2023/인건비/총비용비율', '2023/조세/조세차감전순이익비율',
       '2023/조세공과/총비용비율', '2021/배당율', '2022/배당율', '2023/배당율',
       '2022/부가가치(종업원1인당)', '2023/부가가치(종업원1인당)', '2022/인건비(종업원1인당)',
       '2023/인건비(종업원1인당)', '2023/총자본투자효율', '2023/설비투자효율', '2022/기계투자효율',
       '2023/기계투자효율', '2023/부가가치율', '2023/노동소득분배율', '2023/법인세차감전순이익(구성비)',
       '2023/인건비(구성비)', '2023/금융비용(구성비)', '2023/임차료(구성비)', '2023/조세공과(구성비)',
       '2023/감가상각비(구성비)'],
      dtype='object')

결측 비율이 0.8 이상인 변수들:
Index(['2021/배당율', '2022/배당율', '2023/배당율'], dtype='object')

결측 비율이 0.9 이상인 변수들:
Index([], dtype='object')


결측값에 대해서 중앙값으로 대체

In [464]:
import pandas as pd
from scipy.stats import mstats

# 결측치를 중앙값으로 대체
data = data.fillna(data.median())

# 결측치 대체 결과 확인
print("결측치 대체 결과:")
print(data.isnull().mean())

결측치 대체 결과:
업체코드               0.000000
종목명                0.000033
2021/총자산증가율        0.000000
2022/총자산증가율        0.000000
2023/총자산증가율        0.000000
                     ...   
2021/감가상각비(구성비)    0.000000
2022/감가상각비(구성비)    0.000000
2023/감가상각비(구성비)    0.000000
2023/부실기업          0.000000
상장여부               0.000000
Length: 328, dtype: float64


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """


이상치(Outlier)를 처리하는 방법으로 윈저라이징(Winsorizing)을 적용

In [465]:
# 윈저라이징 적용 (누적확률 0.01, 0.99에 해당하는 값들로 변환)
data_winsorized = data.apply(lambda x: mstats.winsorize(x, limits=[0.01, 0.01]) if x.dtype.kind in 'biufc' else x)

# 결과 확인
print("\n윈저라이징이 적용된 데이터프레임:")
print(data_winsorized.head())


윈저라이징이 적용된 데이터프레임:
      업체코드    종목명  2021/총자산증가율  2022/총자산증가율  2023/총자산증가율  2021/유형자산증가율  \
0  N350605   동화약품         2.70         3.27        10.23         10.41   
1  N320498  KR모터스         1.45        -0.52        -4.89          5.78   
2  N320684     경방         0.15        -6.09        -1.40        -23.56   
3  N320730  삼양홀딩스        22.28        -0.76        17.63        943.89   
4  N310581  하이트진로        10.65        -9.43        -0.03         -2.60   

   2022/유형자산증가율  2023/유형자산증가율  2021/유동자산증가율  2022/유동자산증가율  ...  2022/임차료(구성비)  \
0          7.64         15.13         -3.11          0.95  ...            1.3   
1          4.85        -10.98        -12.71         19.15  ...            1.3   
2        665.18         -4.12         -0.72        -22.21  ...            1.3   
3          0.68          8.08        -15.68         17.97  ...            1.3   
4         -1.51          0.81         55.77        -24.09  ...            1.3   

   2023/임차료(구성비)  2021/조세공과(구성비)  2022/조세공과(구성비)

### 데이터 저장

In [466]:
# CSV 파일로 저장
data.to_csv("../../project/data/clean_financial_data_eda2.csv", index=False, encoding='utf-8-sig')

### 이후 할것 스케치

전처리(변수선택)
- 후진제거법
- t-검정

모델링
- 20,21년도 변수 -> 22년 부실기업 여부 예측
- 20,21,22년도 변수 -> 23년도 부실기업 여부 예측

In [467]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/clean_financial_data_eda2.csv", encoding='utf-8-sig', low_memory=False)

In [468]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30304 entries, 0 to 30303
Data columns (total 328 columns):
 #    Column                     Non-Null Count  Dtype  
---   ------                     --------------  -----  
 0    업체코드                       30304 non-null  object 
 1    종목명                        30303 non-null  object 
 2    2021/총자산증가율                30304 non-null  float64
 3    2022/총자산증가율                30304 non-null  float64
 4    2023/총자산증가율                30304 non-null  float64
 5    2021/유형자산증가율               30304 non-null  float64
 6    2022/유형자산증가율               30304 non-null  float64
 7    2023/유형자산증가율               30304 non-null  float64
 8    2021/유동자산증가율               30304 non-null  float64
 9    2022/유동자산증가율               30304 non-null  float64
 10   2023/유동자산증가율               30304 non-null  float64
 11   2021/재고자산증가율               30304 non-null  float64
 12   2022/재고자산증가율               30304 non-null  float64
 13   2023/재고자산증가율               30

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


t-검정

In [469]:
import pandas as pd
from scipy.stats import ttest_ind

# 종속 변수는 '2023/부실기업'으로 가정
y = data['2023/부실기업']
X = data.drop(columns=['2023/부실기업', '업체코드', '종목명'])

# t-검정을 통해 변수 선택
def t_test_selection(X, y, significance_level=0.05):
    selected_vars = []
    for column in X.columns:
        group1 = X[y == 0][column]
        group2 = X[y == 1][column]
        t_stat, p_value = ttest_ind(group1, group2)
        if p_value < significance_level:
            selected_vars.append(column)
    return selected_vars

# t-검정을 통해 선택된 변수들
selected_vars_ttest = t_test_selection(X, y)

# 선택된 변수들로 새로운 데이터프레임 생성
X_selected_ttest = data[['업체코드', '종목명'] + selected_vars_ttest]

# 종속 변수 y를 데이터프레임에 추가
X_selected_ttest['2023/부실기업'] = y

# 결과 확인
print("\n선택된 변수들로 구성된 데이터프레임:")
print(X_selected_ttest.columns)


선택된 변수들로 구성된 데이터프레임:
Index(['업체코드', '종목명', '2021/총자산증가율', '2022/총자산증가율', '2023/총자산증가율',
       '2022/유동자산증가율', '2022/재고자산증가율', '2021/자기자본증가율', '2023/자기자본증가율',
       '2022/영업이익증가율',
       ...
       '2022/총자본투자효율', '2023/총자본투자효율', '2021/기계투자효율', '2022/기계투자효율',
       '2023/기계투자효율', '2021/부가가치율', '2023/부가가치율', '2021/임차료(구성비)',
       '2022/임차료(구성비)', '2023/부실기업'],
      dtype='object', length=158)


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


후진 선택법

-> 코드 잘 안먹어서 일단 넘어감

In [470]:
# CSV 파일로 저장
X_selected_ttest.to_csv("../../project/data/clean_financial_data_eda3.csv", index=False, encoding='utf-8-sig')

## 모델링

In [471]:
RANDOM_STATE = 110

data = pd.read_csv("../../project/data/clean_financial_data_eda3.csv", encoding='utf-8-sig', low_memory=False)

In [479]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import Adam


In [475]:
# 종속 변수는 '2023/부실기업'으로 가정
y = data['2023/부실기업']
X = data.drop(columns=['2023/부실기업', '업체코드', '종목명'])

# 데이터 분할 (70:30 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [476]:
# 머신러닝 앙상블 모형 설정
# 랜덤포레스트
rf_model = RandomForestClassifier(n_estimators=50, max_depth=5, min_samples_split=10, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")

# 서포트 벡터 머신 앙상블
svm_model = BaggingClassifier(base_estimator=SVC(C=2, kernel='linear'), n_estimators=50, random_state=42)
svm_model.fit(X_train_scaled, y_train)
svm_predictions = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Ensemble Accuracy: {svm_accuracy}")

# K-최근접 이웃법 앙상블
knn_model = BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=30, metric='euclidean'), n_estimators=50, random_state=42)
knn_model.fit(X_train_scaled, y_train)
knn_predictions = knn_model.predict(X_test_scaled)
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f"KNN Ensemble Accuracy: {knn_accuracy}")

Random Forest Accuracy: 0.9575450945886493
SVM Ensemble Accuracy: 0.907391113066432
KNN Ensemble Accuracy: 0.8983721953365597


In [494]:
# 딥러닝 모형 설정

# 합성곱신경망 (CNN)
cnn_model = Sequential()
cnn_model.add(Reshape((X_train_scaled.shape[1], 1, 1), input_shape=(X_train_scaled.shape[1], 1))) 
cnn_model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same'))
cnn_model.add(MaxPooling2D(pool_size=(1, 1)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.25))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# RNN-LSTM
lstm_model = Sequential()
lstm_model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
lstm_model.add(LSTM(128, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# RNN-GRU
gru_model = Sequential()
gru_model.add(GRU(128, activation='relu', return_sequences=True, input_shape=(X_train_scaled.shape[1], 1)))
gru_model.add(GRU(128, activation='relu'))
gru_model.add(Dense(1, activation='sigmoid'))
gru_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [496]:
# 딥러닝 모형 학습 및 평가

# CNN 학습 및 평가
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1, 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1, 1))
cnn_model.fit(X_train_cnn, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test)[1]
print(f"CNN Accuracy: {cnn_accuracy}")

# RNN-LSTM 학습 및 평가
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
lstm_model.fit(X_train_lstm, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test)[1]
print(f"LSTM Accuracy: {lstm_accuracy}")

# RNN-GRU 학습 및 평가
gru_model.fit(X_train_lstm, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
gru_accuracy = gru_model.evaluate(X_test_lstm, y_test)[1]
print(f"GRU Accuracy: {gru_accuracy}")

Epoch 1/200
531/531 [==============================] - 11s 20ms/step - loss: 0.0977 - accuracy: 0.9607 - val_loss: 0.1311 - val_accuracy: 0.9529
Epoch 2/200
531/531 [==============================] - 11s 21ms/step - loss: 0.0903 - accuracy: 0.9631 - val_loss: 0.1016 - val_accuracy: 0.9628
Epoch 3/200
531/531 [==============================] - 11s 21ms/step - loss: 0.0862 - accuracy: 0.9651 - val_loss: 0.1025 - val_accuracy: 0.9651
Epoch 4/200
531/531 [==============================] - 11s 20ms/step - loss: 0.0820 - accuracy: 0.9669 - val_loss: 0.1043 - val_accuracy: 0.9649
Epoch 5/200
531/531 [==============================] - 11s 20ms/step - loss: 0.0830 - accuracy: 0.9669 - val_loss: 0.1123 - val_accuracy: 0.9623
Epoch 6/200
531/531 [==============================] - 10s 20ms/step - loss: 0.0831 - accuracy: 0.9680 - val_loss: 0.1257 - val_accuracy: 0.9628
Epoch 7/200
531/531 [==============================] - 10s 19ms/step - loss: 0.0797 - accuracy: 0.9692 - val_loss: 0.1108 - val_ac

In [497]:
print(f"Deep Learning Models: CNN, LSTM, GRU")
print(f"CNN Accuracy: {cnn_accuracy}")
print(f"LSTM Accuracy: {lstm_accuracy}")
print(f"GRU Accuracy: {gru_accuracy}")

Deep Learning Models: CNN, LSTM, GRU
CNN Accuracy: 0.9580950140953064
LSTM Accuracy: 0.8013638257980347
GRU Accuracy: 0.9439067244529724


.